### Searching exported data from slack
 -- Ben Cardoen, 20181216

* Download the zipped archive from: https://drive.google.com/drive/folders/1LaH2MORhwWK6J92sS66pRw7pOX0EpgNZ
* Unzip the archive in _some-folder_
* Set the variable below to _some-folder_

In [64]:
path = '/home/bcardoen/Downloads/mial/MIAL Slack export Sep 28 2018 - Dec 13 2018'

In [65]:

import json
import os
import pytz
from datetime import datetime, timedelta, timezone
import re
from pprint import pprint

def treedir(pathname):
    (t, d, f)  = next(os.walk(pathname))
    return {**{dir:treedir(os.path.join(pathname, dir)) for dir in d}, **{file:os.path.join(t, file) for file in f}}

def search_channel(channel, regex, path):
    '''
    :param path: a location where the archive is unzipped.
    '''
    contents = None
    try:
        contents = treedir(path)
    except Error as e:
        print('\t Failed to open unzipped in {}'.format(path))
        return
    if channel not in contents:
        print('\t No such channel {}'.format(channel))
        return
    channel_contents = contents[channel]
    found = {}
    tz = pytz.timezone('America/Vancouver')
    for date, datefile in channel_contents.items():
        with open(datefile) as file:
            data = json.load(file)
            for entry in data:
                if entry['type'] == 'message':
                    matches = re.findall(regex, entry['text'])
                    if matches:
                        _time = entry['ts']
                        ts = float(_time)
                        dt = datetime.fromtimestamp(ts, tz)
                        found[str(dt)] = entry['text']
    return found

### Example usage

In [66]:
channelname = 'general'
query='https://*'

outputpath = '.'

results = search_channel(channelname, query, path)

with open(os.path.join(outputpath, 'results.txt'), 'w') as f:
    f.write('Results for query\n {}  in channel \n{}\n'.format(query, channelname))
    for i,(k,v) in enumerate(results.items()):
        f.write('{} th result\n'.format(i))
        f.write('Date \t: {}\n'.format(k))
        f.write('\n')
        f.write('Message \t: {}\n'.format(v))
        f.write('\n')